# <b>Object Detection with AutoML and Vision API</b>
<br>

Learning Objectives

1. Learn how to create and import an image dataset for AutoML Vision
1. Learn how to train AutoML to build an object detection model
1. Learn how to evaluate a model trained with AutoML
1. Learn how to deploy a model trained with AutoML
1. Learn how to predict on new test data with AutoML


## <b>Introduction</b> ##

In this notebook we will use AutoML Vision Object Detection to train a machine learning model that is capable of detecting multiple objects in a given image and provides information about the object and where the object was found in the image.

Before we begin make sure you have created a Project on GCP Console and enabled the AutoML and Cloud Storage APIs
Documentation:
https://cloud.google.com/vision/automl/object-detection/docs/before-you-begin

### <b> Install AutoML package </b> ###
<b>Caution: Run the following command and restart the kernel afterwards.</b>


In [ ]:
pip install google-cloud-automl

<br>

### <b>Set the correct environment variables </b> ###
The following variables should be updated according to your own environment:


In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID" # Replace with your PROJECT ID
SERVICE_ACCOUNT = "demo-920" # Replace with a name of your choice
ZONE = "us-central1"# Make sure the zone is set to "us-central1"


<br>

The following variables are computed from the one you set above, and should not be modified:

In [ ]:
import os

PWD = os.path.abspath(os.path.curdir)

SERVICE_KEY_PATH = os.path.join(PWD, "{0}.json".format(SERVICE_ACCOUNT))
SERVICE_ACCOUNT_EMAIL="{0}@{1}.iam.gserviceaccount.com".format(SERVICE_ACCOUNT, PROJECT_ID)
print(SERVICE_ACCOUNT_EMAIL)
print(PROJECT_ID)

# Exporting the variables into the environment to make them available to all the subsequent cells
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["SERVICE_ACCOUNT"] = SERVICE_ACCOUNT
os.environ["SERVICE_KEY_PATH"] = SERVICE_KEY_PATH
os.environ["SERVICE_ACCOUNT_EMAIL"] = SERVICE_ACCOUNT_EMAIL
os.environ["ZONE"] = ZONE


<br>

### <b>Switching the right project and zone</b> ###

In [ ]:
%%bash
gcloud config set project $PROJECT_ID
gcloud config set compute/region $ZONE


<br>

### <b>Create a service account and generate service key</b> ###

In [ ]:
%%bash
gcloud iam service-accounts list | grep $SERVICE_ACCOUNT ||
gcloud iam service-accounts create $SERVICE_ACCOUNT


In [ ]:
%%bash
test -f $SERVICE_KEY_PATH || 
gcloud iam service-accounts keys create $SERVICE_KEY_PATH \
  --iam-account $SERVICE_ACCOUNT_EMAIL

echo "Service key: $(ls $SERVICE_KEY_PATH)"


<br>

### <b>Make the key available to google clients for authentication</b> ###

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_KEY_PATH

<br>

### <b>Grant service account required role permissions</b> ###

In [ ]:
%%bash

gcloud projects add-iam-policy-binding $PROJECT_ID \
 --member "serviceAccount:$SERVICE_ACCOUNT_EMAIL" \
 --role "roles/automl.admin" \
 --role "roles/storage.admin"


<br>

## <b>Step 1: Create a dataset</b> ##

Create a dataset that will serve as an input for training our model
Documentation:
https://cloud.google.com/vision/automl/object-detection/docs/create-datasets


In [ ]:
from google.cloud import automl

project_id = "your_gcp_project_id" # Replace with your project id
display_name = "dataset_name" # Replace with desired dataset name

client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = client.location_path(project_id, "us-central1")
metadata = automl.types.ImageObjectDetectionDatasetMetadata()
dataset = automl.types.Dataset(
    display_name=display_name,
    image_object_detection_dataset_metadata=metadata,
)

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(project_location, dataset)

created_dataset = response.result()

# Display the dataset information
print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))


<br>

## <b>Step 2: Import images into a dataset</b> ##


After you have created a dataset and obtained the dataset ID, you can import image URIs and from a CSV file stored in a Google Cloud Storage bucket.

Documentation on how to prepare your data and how to create a CSV file for import can be found here:
https://cloud.google.com/vision/automl/object-detection/docs/prepare

In this notebook we will use a publicly available Salads dataset that is located at gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv.

Please note the import might take a couple of minutes to finish depending on the file size


In [ ]:
from google.cloud import automl

dataset_id = "IOD7192557567678087168" # Replace with Dataset id you are given after creating the dataset
path = "gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv" # Replace with desired URI Cloud storage bucket path that contains a CSV file

client = automl.AutoMlClient()
# Get the full path of the dataset.
dataset_full_id = client.dataset_path(
    project_id, "us-central1", dataset_id
)
# Get the multiple Google Cloud Storage URIs
input_uris = path.split(",")
gcs_source = automl.types.GcsSource(input_uris=input_uris)
input_config = automl.types.InputConfig(gcs_source=gcs_source)
# Import data from the input URI
response = client.import_data(dataset_full_id, input_config)

print("Processing import...")
print("Data imported. {}".format(response.result()))


<br>

## <b>Step 3: Train your AutoML Vision model</b> ##

Please note that the AutoML API does not currently include methods for labeling images but it can be inspected in the AutoML UI.

Documentation about annotating training images:
https://cloud.google.com/vision/automl/object-detection/docs/label

Once you are happy with the annotations you can proceed to train the model. Training time takes approximately 1-3h


In [ ]:
from google.cloud import automl

project_id = "your_gcp_project_id" # Replace with your project id
dataset_id = "IOD7192557567678087168" # Replace with Dataset id you are given after creating the dataset
display_name = "model_name" # Replace with desired model name

client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = client.location_path(project_id, "us-central1")
# Leave model unset to use the default base model provided by Google
# train_budget_milli_node_hours: The actual train_cost will be equal or
# less than this value.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#imageobjectdetectionmodelmetadata
metadata = automl.types.ImageObjectDetectionModelMetadata(
    train_budget_milli_node_hours=24000
)
model = automl.types.Model(
    display_name=display_name,
    dataset_id=dataset_id,
    image_object_detection_model_metadata=metadata,
)

# Create a model with the model metadata in the region.
response = client.create_model(project_location, model)

print("Training operation name: {}".format(response.operation.name))
print("Training started...")


<br>

### <b>Information about the trained model</b> ###

In [ ]:
from google.cloud import automl

model_id = "IOD8472955350897655808" # Replace with model_id after you have trained the model

client = automl.AutoMlClient()
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)
model = client.get_model(model_full_id)

# Retrieve deployment state.
if model.deployment_state == automl.enums.Model.DeploymentState.DEPLOYED:
    deployment_state = "deployed"
else:
    deployment_state = "undeployed"

# Display the model information.
print("Model name: {}".format(model.name))
print("Model id: {}".format(model.name.split("/")[-1]))
print("Model display name: {}".format(model.display_name))
print("Model create time:")
print("\tseconds: {}".format(model.create_time.seconds))
print("\tnanos: {}".format(model.create_time.nanos))
print("Model deployment state: {}".format(deployment_state))


<br>

## <b>Step 4: Evaluate the model</b> ##

Once you have trained a model, you can list evaluation metrics for that model.

Please note - Adjusting the confidence threshold and IoU threshold of the model can only be achievend in the UI of AutoML Vision.
You can find more about it here: https://cloud.google.com/vision/automl/object-detection/docs/evaluate


In [ ]:
from google.cloud import automl

project_id = "your_gcp_project_id" # Replace with your project id
model_id = "IOD8472955350897655808" # Replace with model_id after you have trained the model

client = automl.AutoMlClient()
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)

print("List of model evaluations:")
for evaluation in client.list_model_evaluations(model_full_id, ""):
    print("Model evaluation name: {}".format(evaluation.name))
    print(
        "Model annotation spec id: {}".format(
            evaluation.annotation_spec_id
        )
    )
    print("Create Time:")
    print("\tseconds: {}".format(evaluation.create_time.seconds))
    print("\tnanos: {}".format(evaluation.create_time.nanos / 1e9))
    print(
        "Evaluation example count: {}".format(
            evaluation.evaluated_example_count
        )
    )
    print(
        "Object detection model evaluation metrics: {}\n\n".format(
            evaluation.image_object_detection_evaluation_metrics
        )
    )


<br>

## <b>Step 5: Deploy the model</b> ##

In [ ]:
from google.cloud import automl

project_id = "your_gcp_project_id" # Replace with your project id
model_id = "IOD8472955350897655808" # Replace with model_id after you have trained the model

client = automl.AutoMlClient()
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)
response = client.deploy_model(model_full_id)

print("Model deployment finished. {}".format(response.result()))


<br>

## <b>Step 6: Send prediction request</b> ##

In this example we will send an individual prediction for an image that is stored in our project's Cloud storage bucket


In [ ]:
from google.cloud import automl
import tensorflow as tf

project_id = "your_gcp_project_id" # Replace with your project id
model_id = "IOD8472955350897655808" # Replace with model_id after you have trained the model
file_path = "gs://your_bucket_name-vcm/salads/salad-1264107_1920.jpg" # Replace with a Cloud storage bucket uploaded image of your choice

prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = prediction_client.model_path(
    project_id, "us-central1", model_id
)

# Read the file.
with tf.io.gfile.GFile(file_path, "rb") as content_file:
    content = content_file.read()

image = automl.types.Image(image_bytes=content)
payload = automl.types.ExamplePayload(image=image)

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
params = {"score_threshold": "0.8"}

response = prediction_client.predict(model_full_id, payload, params)
print("Prediction results:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print(
        "Predicted class score: {}".format(
            result.image_object_detection.score
        )
    )
    bounding_box = result.image_object_detection.bounding_box
    print("Normalized Vertices:")
    for vertex in bounding_box.normalized_vertices:
        print("\tX: {}, Y: {}".format(vertex.x, vertex.y))
